In [1]:
import tensorflow as tf
import pickle
import numpy as np
import time
import load_cifar_template1
from sklearn.utils import shuffle

C:\Users\ebish\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
#x_train, y_train

<h1>Hyperparameter</h1>

In [49]:
#learning rate
lr = 0.0001
#number of traning steps
num_steps =12
#number of batch_size
batch_size = 256

#network parameters
#hidden_layers=5
n_hidden_1_filters = 6
n_hidden_2_filters = 16
n_hidden_3=120
n_hidden_4=84
num_input = 32
num_classes = 10

In [50]:
tf.reset_default_graph()

<h1>Placeholder</h1>

In [51]:
X = tf.placeholder(tf.float32,[None,num_input,num_input,3],name='X')
Y = tf.placeholder(tf.int32,[None,num_classes],name='Y')

#each filter: f x f x nc(l-1)
#weights: f x f x nc(l-1) x nc(l)
#bias: nc(l)

#Layers weight & bias
weights = {
    'W1_conv1': tf.Variable(tf.random_normal([5,5,3,6]),name='W1_conv1'),
    'W2_conv2': tf.Variable(tf.random_normal([5,5,6,16]),name='W2_conv2'),
    'W3': tf.Variable(tf.random_normal([400, n_hidden_3]),name='W3'),
    'W4': tf.Variable(tf.random_normal([120, n_hidden_4]),name='W4'),
    'Wout': tf.Variable(tf.random_normal([n_hidden_4, num_classes]),name='Wout')
}

biases = {
    'b1_conv1': tf.Variable(tf.zeros(shape=[n_hidden_1_filters]),name='b1_conv1'),
    'b2_conv2': tf.Variable(tf.zeros(shape=[n_hidden_2_filters]),name='b2_conv2'),
    'b3': tf.Variable(tf.zeros(shape=[n_hidden_3]),name='b3'),
    'b4': tf.Variable(tf.zeros(shape=[n_hidden_4]),name='b4'),

    'bout': tf.Variable(tf.zeros(shape=[num_classes]),name='bout')
} 

<h1>LeNet-5</h1>

In [52]:
#define a neural net model
def lenet(x):
    conv_1_out = tf.nn.relu(tf.add(tf.nn.conv2d(x,weights['W1_conv1'],strides=[1,1,1,1],padding='VALID'),biases['b1_conv1']))
    pool_1_out=tf.nn.avg_pool(conv_1_out,ksize=[1,2,2,1], strides=[1,2,2,1],padding='VALID')
    conv_2_out = tf.nn.relu(tf.add(tf.nn.conv2d(pool_1_out,weights['W2_conv2'],strides=[1,1,1,1],padding='VALID'),biases['b2_conv2']))
    pool_2_out=tf.nn.avg_pool(conv_2_out,ksize=[1,2,2,1], strides=[1,2,2,1],padding='VALID')
    pool_2_flatten=tf.reshape(pool_2_out,[-1,5*5*16])
    layer_3_out = tf.nn.relu(tf.add(tf.matmul(pool_2_flatten,weights['W3']),biases['b3']))
    layer_4_out = tf.nn.relu(tf.add(tf.matmul(layer_3_out,weights['W4']),biases['b4']))

    out = tf.add(tf.matmul(layer_4_out,weights['Wout']),biases['bout'])
    return out

<h1>Cost and Optimization</h1>

In [53]:
#predicted labels
logits = lenet(X)

#define loss
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits,labels=Y),name='loss')
#define optimizer
optimizer = tf.train.AdamOptimizer(learning_rate=lr)
train_op = optimizer.minimize(loss)

#compare the predicted labels with true labels
correct_pred = tf.equal(tf.argmax(logits,1),tf.argmax(Y,1))

#compute the accuracy by taking average
accuracy = tf.reduce_mean(tf.cast(correct_pred,tf.float32),name='accuracy')

#Initialize the variables
init = tf.global_variables_initializer()


# loss_summary=tf.summary.scalar('loss',loss)
# accuracy_summary=tf.summary.scalar('accuracy',accuracy)
# #file_writer=tf.summary.FileWriter(logdir,tf.get_default_graph())

<h1>Training, validation and testing</h1>
<h2>Train your model only 10 epochs.</h2>
<h2>1.Print out validation accuracy after each training epoch</h2>
<h2>2.Print out training time for each training epoch</h2>
<h2>3.Print out testing accuracy</h2>

In [54]:
with tf.Session() as sess:
    sess.run(init)
    for i in range(num_steps):
        #fetch batch
        acc=[]
        st=time.time()
        for j in range(1,6):
            for (a,b) in load_cifar_template1.load_preprocessed_training_batch(j,1000):
                batch_x, batch_y = load_cifar_template1.features_reshape(a), b
                #run optimization
                sess.run(train_op, feed_dict={X:batch_x, Y:batch_y})
               
                acc = sess.run(accuracy,feed_dict={X:batch_x, Y:batch_y})
        mid=time.time()
        print("Epoch "+str(i)+", Time:{} ".format(mid-st)+", Accuracy= {:.3f}".format(np.average(acc)))
        
        batch_a,batch_b=load_cifar_template1.load_preprocessed_validation_batch()
        batch_a=load_cifar_template1.features_reshape(batch_a)
        val=sess.run(accuracy, feed_dict={X:batch_a, Y:batch_b})
        print("Validation Accuracy:",val)
    
    print("Training finished!")
    batch_a,batch_b=load_cifar_template1.load_preprocessed_test_batch()
    batch_a=load_cifar_template1.features_reshape(batch_a)
    test_acc=sess.run(accuracy, feed_dict={X:batch_a, Y:batch_b})
    print("Testing Accuracy:",test_acc)
#file_writer.close()

Epoch 0, Time:66.83464860916138 , Accuracy= 0.093
Validation Accuracy: 0.1008
Epoch 1, Time:66.31645178794861 , Accuracy= 0.079
Validation Accuracy: 0.1066
Epoch 2, Time:66.30547070503235 , Accuracy= 0.107
Validation Accuracy: 0.1082
Epoch 3, Time:66.02772092819214 , Accuracy= 0.105
Validation Accuracy: 0.114
Epoch 4, Time:65.03082919120789 , Accuracy= 0.093
Validation Accuracy: 0.11
Epoch 5, Time:67.84182262420654 , Accuracy= 0.130
Validation Accuracy: 0.1172
Epoch 6, Time:64.98388123512268 , Accuracy= 0.133
Validation Accuracy: 0.1308
Epoch 7, Time:66.76444864273071 , Accuracy= 0.153
Validation Accuracy: 0.135
Epoch 8, Time:65.55088233947754 , Accuracy= 0.141
Validation Accuracy: 0.141
Epoch 9, Time:64.08248543739319 , Accuracy= 0.145
Validation Accuracy: 0.1508
Epoch 10, Time:66.68687057495117 , Accuracy= 0.167
Validation Accuracy: 0.1538
Epoch 11, Time:66.03376698493958 , Accuracy= 0.153
Validation Accuracy: 0.1588
Training finished!
Testing Accuracy: 0.1535


# Insights

With this one, I tweaked the learning rate from .1 to .0001 to get us close to 60% accuracy at around 10 training epochs, but I never could seem to get above 15% accuracy on the testing dataset. Not sure why this is, but since we were more focused on the implementation of the framework I didn't worry about it as much